In [30]:
#In this code we can get the word-count of data starting from year 2009.
#we also get the average and standard deviation of words in the interval of 2009-2012, 2013- 2016 and 2017-2020
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from statistics import stdev
import re
import urllib.request
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords =stopwords.words('english')

#here we are creating a new spark session
spark = SparkSession.builder\
                    .master("local")\
                    .appName('FirstAssignment1part1')\
                    .getOrCreate()
sc=spark.sparkContext

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
}

#loading the website and its contents
url = "http://stateoftheunion.onetwothree.net/texts/index.html"
requ = urllib.request.Request(url,headers=headers)
req_url = urllib.request.urlopen(requ) 
contents = req_url.read()
contents = str(contents)

#writing the contents into alldata file.
f = open('alldata.txt', 'w')
f.writelines(contents)
f.close()
f = open('alldata.txt','r')
contents = f.read()
h_r = r'href=[\'"]?([^\'" >]+)'
pagenames = re.findall(h_r, contents)
allnames = list()

#creating and saving the contents of each link into thier specific txt file such as contentfile-17901208.html.txt
for index,pagename in enumerate(pagenames):
    if pagename[0:2] == '17' or pagename[0:2] == '18' or pagename[0:2] == '19' or pagename[0:2] == '20':
        allnames.append(pagename)
        file = open('contentfile-%s.txt' % pagename, 'w')
        url = "http://stateoftheunion.onetwothree.net/texts/"+pagename
        req = urllib.request.Request(url,headers=headers)
        request_url = urllib.request.urlopen(req) 
        contents = request_url.read()
        file.write(str(contents))      

#here we are removing the punctuation, html, or stopwords and removing them
def RemoveNT(a):
    a = a.replace("\\n", " ")
    a = a.replace("\\t", " ")
    return a

def RemovePunc(a):
    punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
    lowcase = a.lower()
    for ch in punc:
        lowcase = lowcase.replace(ch, " ")
    return lowcase

html_words = ["href","div","index","style","tabs","cell","html","link",
              "src","script","image","center","alt","html","xml","version","meta"
              "css","http","type","script","class","text","instance"]
def RemoveHtml(a):
    for word in html_words:
        if a.count(word) > 0:
            a = " "
    return a
        
Tags = re.compile(r'<[^>]+>')
def RemoveTags(a):
    return Tags.sub("", a)

def RemoveStopw(a):
    return " ".join([word for word in str(a).split() if word not in stopwords])

def RemoveDigit(a):
    res = any(chr.isdigit() for chr in a)
    if res:
        a = " "
    return a

my_rdd = sc.emptyRDD()
my_rdd_list = list()
count = 0
a=0
alllinksnew = allnames[222:] #this starts the data to come starting from 2009
for i in range(3):
    alllinktemp = alllinksnew[a:4+a]#so seperate data into years 2009-2012, 2013- 2016 and 2017-2020
    for index,pagename in enumerate(alllinktemp):
        filename = "contentfile-"+pagename+".txt"
        my_rdd=sc.textFile(filename)
        my_rdd=my_rdd.flatMap(lambda satir: satir.split(" "))
        my_rdd = my_rdd.map(RemoveNT)
        my_rdd = my_rdd.map(RemovePunc)
        my_rdd = my_rdd.map(RemoveHtml)
        my_rdd = my_rdd.map(RemoveTags)
        my_rdd = my_rdd.map(RemoveStopw)
        my_rdd = my_rdd.map(RemoveDigit)
        my_rdd=my_rdd.flatMap(lambda satir: satir.split(" "))
        my_rdd = my_rdd.filter(lambda x:x!='')
        wordCount=my_rdd.map(lambda  word:(word,1))
        wordCount = wordCount.reduceByKey(lambda x,y:(x+y)).sortByKey()
        print(filename," word count is:",wordCount.count())
        my_rdd_list.append(wordCount)
    rdd=0
    rdd2=0
    rdd = sc.union(my_rdd_list)
    
    rdd2 = rdd.reduceByKey(lambda x,y:(x+y)).sortByKey()
    #for average we have divided by 4 as there are 4 year groups
    rdd2 = rdd2.mapValues(lambda x: x / 4)
    rdd2 = rdd2.sortBy(lambda x:x[1],ascending=False)
    #here we are print the average of each word
    print("The average of each word is as follows")
    print(rdd2.take(10))
  
    
    y = rdd.groupByKey()
    y = y.sortByKey()
    # this creates a calculation of standard deviation with result as false for those whose len is less than 1 
    y1 = y.mapValues(lambda x: len(x)>1 and stdev(x))
    y1 = y1.sortBy(lambda x:x[1],ascending=False)
    print("The standard deviation of each word is as follows")
    print(y1.take(10))
    
    avg_sd_list = list()
    y_prev=y
    for t in y.collect():
        if len(t[1]) > 1:
            #here we are storing the average plus twice the standard deviation 
            avg_sd_list.append((t[0],(sum(t[1])/4) +(2 * stdev(t[1]))))
    avg_sd_rdd = sc.parallelize(avg_sd_list)
    #here we print the average and twice standard deviation of each word if needed
    #print("The average of twice standard deviation of each word is as follows")
    #print(avg_sd_rdd.collect())
    
    #here we compute the data for the year following the window such as year 2013 for window 2009-2012
    my_rdd2 = sc.emptyRDD()
    my_rdd2_list = list()
    alllinktemp2 = alllinksnew[4+a]
    print(alllinktemp2)
    index=0
    pagename=0
    filename1 = "contentfile-"+ alllinktemp2 +".txt"
    print("the next year following the window is as follows")
    print(filename1)
    my_rdd2 = sc.textFile(filename1)
    my_rdd2 = my_rdd2.flatMap(lambda satir: satir.split(" "))
    my_rdd2 = my_rdd2.map(RemoveNT)
    my_rdd2 = my_rdd2.map(RemovePunc)
    my_rdd2 = my_rdd2.map(RemoveHtml)
    my_rdd2 = my_rdd2.map(RemoveTags)
    my_rdd2 = my_rdd2.map(RemoveStopw)
    my_rdd2 = my_rdd2.map(RemoveDigit)
    my_rdd2 = my_rdd2.flatMap(lambda satir: satir.split(" "))
    my_rdd2 = my_rdd2.filter(lambda x:x!='')
    word_count2 = my_rdd2.map(lambda  word:(word,1))
    word_count2 = word_count2.reduceByKey(lambda x,y:(x+y)).sortByKey()
    #print(word_count2.count())
    my_rdd2_list.append(word_count2) 
    rdd2=0
    rdd2 = sc.union(my_rdd2_list)
    rdd2.collect()
    y2 = rdd2.groupByKey()
    for t in y2.collect():
        t1=0
        t1=avg_sd_rdd.lookup(t[0])
        if(sum(t1)<sum(t[1]) and sum(t1)!=0):
            print("the frequency word exceeding the average plus two standard deviations is :-",t[0])
    a+=4

# To stop the current spark session
sc.stop()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Madhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


contentfile-20090224.html.txt  word count is: 1359
contentfile-20100127.html.txt  word count is: 1551
contentfile-20110125.html.txt  word count is: 1544
contentfile-20120124.html.txt  word count is: 1599
The average of each word is as follows
The standard deviation of each word is as follows
20130212.html
the next year following the window is as follows
contentfile-20130212.html.txt
the frequency word exceeding the average plus two standard deviations is :- absolutely
the frequency word exceeding the average plus two standard deviations is :- accept
the frequency word exceeding the average plus two standard deviations is :- achieve
the frequency word exceeding the average plus two standard deviations is :- added
the frequency word exceeding the average plus two standard deviations is :- affiliates
the frequency word exceeding the average plus two standard deviations is :- agenda
the frequency word exceeding the average plus two standard deviations is :- ago
the frequency word exceeding

KeyboardInterrupt: 

In [32]:
#This is the part 2 of the assignment.
#in this part we calculate the frequency of co-occurence of words , Output 20 frequent pairs of words and lift between two words
#and output those whose lift is bigger than 3.0
import findspark
findspark.init()
import re
from itertools import islice
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from collections import Counter, OrderedDict
import urllib.request
from statistics import stdev
import re
import urllib.request
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords =stopwords.words('english')


#here we are creating a new spark session
spark = SparkSession.builder\
                    .master("local")\
                    .appName('FirstAssignment1part2')\
                    .getOrCreate()
sc=spark.sparkContext


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
}


#loading the website and its contents
url = "http://stateoftheunion.onetwothree.net/texts/index.html"
requ = urllib.request.Request(url,headers=headers)
#To obtain the source code of the website
req_url = urllib.request.urlopen(requ) 
contents = req_url.read()
contents = str(contents)

#writing the contents into alldata file.
f = open('alldata.txt', 'w')
f.writelines(contents)
f.close()
f = open('alldata.txt','r')
contents = f.read()
href_regex = r'href=[\'"]?([^\'" >]+)'
pagenames = re.findall(href_regex, contents)
allnames = list()

#creating and saving the contents of each link into thier specific txt file such as contentfile-17901208.html.txt
for index,pagename in enumerate(pagenames):
    if pagename[0:2] == '17' or pagename[0:2] == '18' or pagename[0:2] == '19' or pagename[0:2] == '20':
        allnames.append(pagename)
        file = open('contentfile-%s.txt' % pagename, 'w')
        url = "http://stateoftheunion.onetwothree.net/texts/"+pagename
        req = urllib.request.Request(url,headers=headers)
        #To obtain the source code of the website
        request_url = urllib.request.urlopen(req) 
        contents = request_url.read()
        file.write(str(contents))    

#here we are removing the punctuation, html, or stopwords and removing them
def RemoveNT(a):
    a = a.replace("\\n", " ")
    a = a.replace("\\t", " ")
    return a

def RemovePunc(a):
    punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
    lowcase = a.lower()
    for ch in punc:
        lowcase = lowcase.replace(ch, " ")
    return lowcase

html_words = ["href","div","index","style","tabs","cell","html","link",
              "src","script","image","center","alt","html","xml","version","meta"
              "css","http","type","script","class","text","instance","p"]
def RemoveHtml(a):
    for word in html_words:
        if a.count(word) > 0:
            a = " "
    return a
        
Tags = re.compile(r'<[^>]+>')
def RemoveTags(a):
    return Tags.sub("", a)

def RemoveStopw(a):
    return " ".join([word for word in str(a).split() if word not in stopwords])

def RemoveDigit(a):
    res = any(chr.isdigit() for chr in a)
    if res:
        a = " "
    return a

my_rdd = sc.emptyRDD()
my_rdd2 = sc.emptyRDD()
my_rdd_list = list()
my_count_list = list()
count = 0
alllinksnew = allnames
import itertools
for index,pagename in enumerate(alllinksnew):
    filename = "contentfile-"+pagename+".txt"
    print(filename)
    my_rdd=sc.textFile(filename)
    my_rdd=my_rdd.flatMap(lambda satir: satir.split("."))
    my_rdd2 = my_rdd.map(RemoveNT)
    my_rdd2 = my_rdd2.map(RemovePunc)
    my_rdd2 = my_rdd2.map(RemoveHtml)
    my_rdd2 = my_rdd2.map(RemoveTags)
    my_rdd2 = my_rdd2.map(RemoveStopw)
    my_rdd2 = my_rdd2.map(RemoveDigit)
    my_rdd2 = my_rdd2.filter(lambda x:x!='')
    my_rdd2 = my_rdd2.filter(lambda x:x!=' ')
    my_rdd_list.append(my_rdd2)

rdd = sc.union(my_rdd_list)
final = (rdd.filter(lambda x: x != "").map(lambda x: x.split(" "))
        .flatMap(lambda x: itertools.combinations(x, 2))
        .filter(lambda x: x[0] != "")
        .map(lambda x: (x, 1))
        .reduceByKey(lambda x, y: x + y)
        )
final1=final.collect()

final=final.filter(lambda KeyValue: KeyValue[1]>10)

#here we are printing all the frequent pairs above value 10
print("printing all the frequent pairs above value 10")
print(final.collect())

#here we are printing 20 frequent pairs of words
print("printing 20 frequent word-pairs")
print(final.take(20))
rdd2 = rdd.flatMap(lambda satir: satir.split(" "))
rdd2 = rdd2.map(lambda  word:(word,1))
rdd2 = rdd2.reduceByKey(lambda x,y:(x+y)).sortByKey()
my_count_list=rdd2.collect()

#calculating the lift of the pairs greater than 3
lift=0
for m in final1:
    i=j=0
    for n in my_count_list:
        if m[0][0] == n[0]:
            i= n[1]
        if m[0][1] == n[0]:
            j= n[1]
            j=j/100
    if(i!=0 and j!=0):
        lift=(m[1]/(i*j))
    if(lift>3):
        print("the lift greater than 3 for the word",m[0], "is", lift)


# To stop the current spark session
sc.stop()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Madhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


contentfile-17900108.html.txt
contentfile-17901208.html.txt
contentfile-17911025.html.txt
contentfile-17921106.html.txt
contentfile-17931203.html.txt
contentfile-17941119.html.txt
contentfile-17951208.html.txt
contentfile-17961207.html.txt
contentfile-17971122.html.txt
contentfile-17981208.html.txt
contentfile-17991203.html.txt
contentfile-18001111.html.txt
contentfile-18011208.html.txt
contentfile-18021215.html.txt
contentfile-18031017.html.txt
contentfile-18041108.html.txt
contentfile-18051203.html.txt
contentfile-18061202.html.txt
contentfile-18071027.html.txt
contentfile-18081108.html.txt
contentfile-18091129.html.txt
contentfile-18101205.html.txt
contentfile-18111105.html.txt
contentfile-18121104.html.txt
contentfile-18131207.html.txt
contentfile-18140920.html.txt
contentfile-18151205.html.txt
contentfile-18161203.html.txt
contentfile-18171212.html.txt
contentfile-18181116.html.txt
contentfile-18191207.html.txt
contentfile-18201114.html.txt
contentfile-18211203.html.txt
contentfil